In [ ]:
!pip uninstall -qqy jupyterlab
!pip install -U -q "google-genai==1.7.0"
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

In [ ]:
!pip install -qU "chromadb==0.6.3"

## Declaring required libraies

In [ ]:
from google import genai
from google.genai import types
from IPython.display import Image,display,HTML,Markdown
import numpy as np
import pandas as pd
from kaggle_secrets import UserSecretsClient
import re,json

In [ ]:
GOOGLE_API_KEY = UserSecretsClient().get_secret("Capstone_API_KEY")

## Data Ingestion

    

*1. We are ingesting the data in structured and unstructured data.      
 2. Structured data - SQL and unstructured data - Documents*



## 1. SQL Data

In [ ]:
%load_ext sql
%sql sqlite:///job_profiles.db

In [ ]:
import sqlite3

db_file = "job_profiles.db"
db_conn = sqlite3.connect(db_file)

In [ ]:
%%sql
-- Creating job profiles table where it contains the job role, years of experience and skills

CREATE TABLE IF NOT EXISTS jobProfiles(
    role VARCHAR(255) NOT NULL,
    skills TEXT
) 

In [ ]:
%%sql
INSERT INTO jobProfiles(role, skills) 
VALUES 
  ('GenAI Engineer', 'AI, ML, Python'),
  ('Data Scientist', 'Python, Statistics, SQL, Machine Learning'),
  ('MLOps Engineer', 'Docker, Kubernetes, CI/CD, TensorFlow Serving'),
  ('Python Developer','Python');

## Defining tools for SQL

In [ ]:
def delete_table(table_name: str):
    cursor = db_conn.cursor()
    """Deleting a table from the database."""
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    db_conn.commit()
    return f"Table '{table_name}' has been deleted."  
def describe_table(table_name: str):
    cursor = db_conn.cursor()
    print(f' - DB CALL: describe_table({table_name})')
    cursor.execute(f"PRAGMA table_info({table_name});")
    # cursor.execute(f"DESCRIBE {table_name}");
    schema = cursor.fetchall()
    return [(col[1], col[2]) for col in schema]

def list_tables():
    print(' - DB CALL: list_tables()')
    cursor = db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    return [t[0] for t in tables]

def execute_query(sqlQuery: str) -> str:
    """Execute a SQL query on the database."""
    print(f' - DB CALL: execute_query({sqlQuery})')
    cursor = db_conn.cursor()
    cursor.execute(sqlQuery)
    return str(cursor.fetchall())
# delete_table("jobProfiles")
# describe_table("jobProfiles")
# list_tables()
# execute_query("SELECT * from jobProfiles")

## LLM Prompt for SQL

In [ ]:
## SQL Data Retrieval
db_tools = [describe_table,list_tables,execute_query]

sql_prompt = """
You are a helpful chatbot for a SQL database.

Your task is to understand user questions, run SQL queries using available tools, and provide answers based on the data.

If a question is complex or multi-part, answer what you can using the data. If only partial info is available, 
respond with what’s found — don’t mention what’s missing.

Tools:
- execute_query: Run SQL queries. 
- describe_table: View table schema
- list_tables: View table names

Given a user-provided role, check if the core keywords from the input are present anywhere within the full role names 
stored in our database. The match does not need to be exact. For example, if the user provides 'Gen AI', and the database 
has 'Gen AI Engineer', consider it a match. Use case-insensitive partial matching and focus on keyword presence 
rather than exact string equality.

Respond in this format:
{ "response": "your answer here" }

If nothing is found:
{ "response": "I am unable to find that information" }

Example:
User: What are the skills required for GenAI engineer and explain me what is GenAI?

Response:
{ "response": "Skills required for GenAI Engineer are AI, ML, Python." }
"""

## Defining the LLM Model 

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sql_prompt,
        tools=db_tools,
    ),
)

## Post-processing SQL output format

In [ ]:
def post_processing_SQL_response(response):
    raw_text = response.candidates[0].content.parts[0].text
    print(f"Response from the SQL database: {raw_text}")
    match = re.search(r'\{.*\}', raw_text)
    if match:
        json_part = match.group(0)
        data = json.loads(json_part)
        return data["response"]
    return "Not able to get response. Try debugging the issue...."

## Validating the SQL info retrieval

In [ ]:
resp = chat.send_message("What is RAG?")
result = post_processing_SQL_response(resp)
print(result)

## 2.Text Document

## Declaring documents

In [ ]:
DOCUMENT1 = """
Interview questions for AI:

Q1. What is Artificial Intelligence?
A1. Artificial Intelligence is the simulation of human intelligence in machines that are programmed to think and learn.

Q2. What are the main types of AI?
A2. The types are: Reactive Machines, Limited Memory, Theory of Mind, and Self-aware AI.

Q3. What is the difference between AI, ML, and Deep Learning?
A3. AI is the broader concept of machines being intelligent, ML is a subset of AI focused on learning from data, and Deep Learning is a subset of ML using neural networks.

Q4. What are some real-world applications of AI?
A4. Examples include voice assistants (e.g. Siri), recommendation systems, autonomous vehicles, and chatbots.

Q5. What is the Turing Test?
A5. The Turing Test is a way to evaluate a machine’s ability to exhibit human-like intelligence.

Q6. What is the difference between Narrow AI and General AI?
A6. Narrow AI is designed for specific tasks, while General AI has the ability to perform any intellectual task that a human can.

Q7. What are the main components of an AI system?
A7. Components include learning, reasoning, problem-solving, perception, and language understanding.

Q8. What is an intelligent agent in AI?
A8. An intelligent agent is an autonomous entity which observes its environment and acts upon it to achieve goals.

Q9. Explain the concept of state space in AI.
A9. State space is the set of all possible states that can be reached in solving a problem.

Q10. What is a heuristic function?
A10. A heuristic function estimates the cost from the current state to the goal, guiding search algorithms.

"""
DOCUMENT2 = """

Interview questions for ML:

Q1. What is Machine Learning?
A1. Machine Learning is a subset of AI that enables machines to learn from data and make decisions without being explicitly programmed.

Q2. What are the types of Machine Learning?
A2. Supervised Learning, Unsupervised Learning, and Reinforcement Learning.

Q3. What is the difference between Supervised and Unsupervised learning?
A3. Supervised learning uses labeled data, while unsupervised learning uses unlabeled data to find hidden patterns.

Q4. What is overfitting and underfitting?
A4. Overfitting is when the model learns noise instead of the signal. Underfitting is when the model is too simple to capture the data's complexity.

Q5. What is the bias-variance tradeoff?
A5. It’s the balance between a model’s ability to generalize (low variance) and its accuracy (low bias).

Q6. What are features and labels in ML?
A6. Features are input variables, and labels are the output (target) variables the model learns to predict.

Q7. What are precision, recall, and F1 score?
A7. Precision: correct positives / predicted positives, Recall: correct positives / actual positives, F1 Score: harmonic mean of precision and recall.

Q8. What is cross-validation?
A8. Cross-validation is a technique to assess model performance by splitting the dataset into multiple training and testing sets.

Q9. What is the purpose of the train/test split?
A9. To evaluate model performance on unseen data by training on one subset and testing on another.

Q10. Name a few popular ML algorithms.
A10. Linear Regression, Logistic Regression, Decision Trees, SVM, K-Nearest Neighbors, Random Forests.
"""
DOCUMENT3 = """

Interview questions for Python:

Q1. What are Python's key features?
A1. Easy syntax, interpreted, dynamically typed, supports OOP, and has vast libraries.

Q2. What is the difference between a list and a tuple?
A2. Lists are mutable (changeable), while tuples are immutable.

Q3. What are Python's data types?
A3. Common types include int, float, str, list, tuple, dict, set, bool.

Q4. What is the difference between `is` and `==`?
A4. `is` checks identity (memory location), `==` checks value equality.

Q5. What are *args and **kwargs?
A5. *args allows variable number of positional arguments, **kwargs allows variable number of keyword arguments.

Q6. What are Python decorators?
A6. Decorators are functions that modify the behavior of other functions without changing their code.

Q7. How is memory managed in Python?
A7. Python uses automatic memory management via reference counting and garbage collection.

Q8. What is a lambda function?
A8. A lambda is an anonymous, one-line function defined using the keyword `lambda`.

Q9. What is the use of `self` in Python classes?
A9. `self` refers to the current instance of the class and is used to access its variables and methods.

Q10. What are list comprehensions?
A10. List comprehensions provide a concise way to create lists using a single line of code.
"""

## Combining all docs in one document

In [ ]:
topics = ["ai", "ml", "python"]
docs = {
    "ai": DOCUMENT1,
    "ml": DOCUMENT2,
    "python": DOCUMENT3
}

## Declaring nessesary libraries

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.genai import types
import chromadb
import json
import re

## Converting documents into embeddings

In [ ]:
## Document Data Retrieval
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
client = genai.Client(api_key=GOOGLE_API_KEY)

class GeminiEmbeddingFunction(EmbeddingFunction):
    isDocumentEmbeddings = True
    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.isDocumentEmbeddings:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

## Storing embeddings in vector databases.

In [ ]:
DB_NAME = "googlecardb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.isDocumentEmbeddings = True
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)
# all_ids = db.get()['ids']
# db.delete(ids=all_ids)

db.add(
    documents=[docs[t] for t in topics],
    ids=topics,
    metadatas=[
    {"topic": "ai"},
    {"topic": "ml"},
    {"topic": "python"}
  ]
)

db.count()
# print(db.get()['ids'])

## Documents Retrieval

In [ ]:
def detect_topic(user_query):
    topics = ["python", "ai", "ml"]
    return [topic for topic in topics if topic in user_query.lower()]
def retrieval_documents(query,threshold=0.8,sql_answer=""):
    print(threshold)
    embed_fn.isDocumentEmbeddings = False
    topics = detect_topic(f"Query:{query}. SQL Response:{sql_answer}")
    print(topics)
    final_results = []
    for topic in topics:
        result = db.query(
            query_texts=[query],
            n_results=3,
            where={"topic": topic}  # Assumes you added metadata: {"topic": "python"} etc.
        )
        final_results.extend(zip(result['documents'][0], result['distances'][0]))
    # print(final_results)
    filtered_results = [(doc, dist) for doc, dist in final_results if dist <= threshold]
    # print(filtered_results)
    return filtered_results

## Generate LLM Response

In [ ]:
def get_llm_response(query,retrieved_documents,sql_answer):
    if sql_answer == "I am unable to find that information" and not retrieved_documents:
        return "No relevant documents found for your query."
    retrieved_documents.sort(key=lambda x: x[1])
    res = "\n\n".join(doc for doc, _ in retrieved_documents)
    prompt = f"""
        You are a helpful and informative assistant that answers user questions by combining insights from structured SQL data and unstructured document embeddings.
        
        Your goal is to provide a clear, complete, and friendly response using the following context:
        - SQL Info (if available): Use it to extract concrete, structured facts.
        - Embedding Search Results (cosine similarity): Use them to supplement or enhance the answer with richer context.
        - If the data seems partial or only part of the question is answered, still try to provide a helpful response based on available context.
        - Your audience is non-technical. Keep your tone friendly and easy to understand, breaking down any technical jargon.
        
        QUESTION:
        {query}
        
        SQL INFO:
        {sql_answer}
        
        DOCUMENTS FROM COSINE SIMILARITY SEARCH:
        {retrieved_documents}
        
        Now, using the above context, provide a complete and concise answer.
        If you have enough information, include examples or details from the documents.
        If the SQL result already answers the question well, summarize or explain it clearly.
        
        Always return your final answer in this format:
        {{ "response" : "your answer here" }}
        """
    answer = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt)
    res = answer.text
    res = re.sub(r"^```json\s*|\s*```$", "", res.strip(), flags=re.IGNORECASE | re.MULTILINE)
    try:
        if res:
            json_obj = json.loads(res)
            return json_obj["response"]
        else:
            print("Empty response.")
    except json.JSONDecodeError as e:
        print("Invalid JSON:", e)
        print("Raw cleaned response:", repr(res))
    # res = answer.text
    # try:
    #     if res.strip(): 
    #         json_obj = json.loads(res)
    #         return json_obj["response"]
    #     else:
    #         print("Empty response text.")
    # except json.JSONDecodeError as e:
    #     print("Invalid JSON:", e)
    return res


## Declaring Langraph libraries

In [ ]:
from google import genai
from google.genai import types
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

## Defining the state schema

In [ ]:
#Define the state 
class state_schema(TypedDict):
    messages : Annotated[list,add_messages]
    query : str
    sql_response: str
    retrieved_documents: list[str]
    final_answer : str
    finished : bool

welcome_msg = "Welcome to the Interview Prep regarding the job role."

## Declaring functions for the Nodes

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph,START,END
from langchain_core.messages.ai import AIMessage

llm = ChatGoogleGenerativeAI(model ="gemini-2.0-flash")

#define the functions 
def conditional_edge(state_schema) -> state_schema:
    if state_schema.get("finished", True):
        return END  
    return "input_query"   
def input_query(state_schema) -> state_schema:  
    if not state_schema["messages"]:
        state_schema["query"] = ""
        state_schema["sql_response"] = ""
        state_schema["retrieved_documents"] = []
        state_schema["final_answer"] = ""
        state_schema["finished"] = False
        state_schema["messages"] = [AIMessage(content=welcome_msg)]  
        print(f"Bot: {welcome_msg}")
    return state_schema
def sql_info(state_schema) -> state_schema:
    if state_schema["messages"]:
        if state_schema["final_answer"] != "":
            print("Bot:",state_schema["final_answer"])
        user_input = input("User: ")
        state_schema["query"] = user_input
        exit_words = {"q", "quit", "exit", "goodbye"}
        print(f"Exit validation: {user_input.strip().lower() in exit_words }")
        if user_input.strip().lower() in exit_words:
            print("user wants to exit from the chat.")
            state_schema["finished"] = True
            return state_schema
        state_schema["messages"] = [("user", user_input)]
        resp = chat.send_message(user_input)
        result = post_processing_SQL_response(resp)
        # print(result)
        state_schema["sql_response"] = result
        state_schema["messages"] = [("ai",result)] 
    return state_schema 
    
def retrieve_documents(state_schema) -> state_schema:
    query = state_schema["query"]
    sql_response = state_schema["sql_response"]
    if sql_response == "I am unable to find that information":
        retrieved_documents = retrieval_documents(query)    
    else:
        retrieved_documents = retrieval_documents(query,0.96,state_schema["sql_response"])
    state_schema["retrieved_documents"] = retrieved_documents
    # print("*"*20)
    # print(f"final result: {final_result}")
    # print("*"*20)
    return state_schema
def get_final_result(state_schema) -> state_schema:
    query = state_schema["query"]
    sql_response = state_schema["sql_response"]
    retrieved_documents = state_schema["retrieved_documents"]
    result = get_llm_response(query,retrieved_documents,sql_response)
    state_schema["final_answer"] = result
    return state_schema

## Build graph and mention the nodes and edges

In [ ]:
graph = StateGraph(state_schema)
graph.add_node("input_query",input_query)
graph.add_node("sql_info",sql_info)
graph.add_node("retrieve_documents",retrieve_documents)
graph.add_node("get_final_result",get_final_result)

graph.add_edge(START,"input_query")
graph.add_edge("input_query","sql_info")
graph.add_edge("sql_info","retrieve_documents")
graph.add_edge("retrieve_documents","get_final_result")
graph.add_conditional_edges("get_final_result", conditional_edge)

chatGraph = graph.compile()

## Display the graph

In [ ]:
from IPython.display import display,Image
# Image(chatGraph.get_graph().draw_mermaid_png())
print(chatGraph.get_graph().draw_mermaid())

## Initializing the chatbot

In [ ]:
state = chatGraph.invoke({"messages": []})